# Inspect training

In [ ]:
from dataclasses import asdict

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import optim

import dataset.mapping

In [ ]:
from dataset.datasets import get_dataset
from settings import TrainingSettings, AnnotationSettings, AudioProcessingSettings, CNNSettings, EvaluationSettings, \
    DatasetSettings
from main import train_epoch, evaluate
from model.cnn import CNN

In [ ]:
training_settings: TrainingSettings = TrainingSettings(epochs=30, scheduler=False, num_workers=16)
audio_settings: AudioProcessingSettings = AudioProcessingSettings()
annotation_settings: AnnotationSettings = AnnotationSettings(time_shift=0.0)
dataset_settings = DatasetSettings(
    audio_settings=audio_settings,
    annotation_settings=annotation_settings,
)
cnn_settings = CNNSettings()
evaluation_settings = EvaluationSettings()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import color_sequences

print(color_sequences)

colors = np.array([
    [230, 25, 75],
    [60, 180, 75],
    [255, 225, 25],
    [0, 130, 200],
    [245, 130, 48],
    [145, 30, 180],
    [70, 240, 240],
    [240, 50, 230],
    [210, 245, 60],
    [250, 190, 190],
    [0, 128, 128],
    [230, 190, 255],
    [170, 110, 40],
    [255, 250, 200],
    [128, 0, 0],
    [170, 255, 195],
    [128, 128, 0],
    [255, 215, 180],
    [0, 0, 128],
    [128, 128, 128],
    [0, 0, 0],
]) / 255
colormap = ListedColormap(colors, N=21)

# Plot a pseudo pr curve
for i in range(10):
    x = np.linspace(0, 1, 100)
    y = np.sin(x * np.pi * i) * 0.5 + 0.5
    plt.plot(x, y, color=colors[i], label=f"Class {i}")
plt.legend()
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall curve")

In [ ]:
loader_train, loader_val, loader_test_rbma, loader_test_mdb = get_dataset(
    training_settings, audio_settings, annotation_settings
)

In [ ]:
model = CNN(**asdict(cnn_settings))
model.to(device)

In [ ]:
max_lr = training_settings.learning_rate
initial_lr = max_lr / 25
_min_lr = initial_lr / 1e4
initial_lr = (
    training_settings.learning_rate
    if not training_settings.scheduler
    else initial_lr
)
optimizer = optim.RAdam(
    model.parameters(), lr=initial_lr, weight_decay=1e-5
)
scheduler = (
    optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=max_lr,
        steps_per_epoch=len(loader_train),
        epochs=training_settings.epochs,
    )
    if training_settings.scheduler
    else None
)
error = torch.nn.BCEWithLogitsLoss(reduction="none")
scaler = torch.cuda.amp.GradScaler()

In [ ]:
model.to(device)
best_loss = float("inf")
best_score = 0
last_improvement = 0
for epoch in range(10):
    train_loss = train_epoch(
        epoch,
        loader_train,
        device,
        None,
        error,
        model,
        optimizer,
        scaler,
        scheduler,
        tensorboard_writer=None,
    )
    val_loss, f_score, avg_f_score = evaluate(
        epoch,
        model,
        loader_val,
        error,
        device,
        evaluation_settings.ignore_beats,
        tensorboard_writer=None,
        tag="Validation",
    )
    print(
        f"Epoch: {epoch + 1} "
        f"Loss: {train_loss * 100:.4f}\t "
        f"Val Loss: {val_loss * 100:.4f} F-Score: {avg_f_score * 100:.4f}/{f_score * 100:.4f}"
    )

In [ ]:
torch.cuda.empty_cache()
model.cpu()
loader = iter(loader_test_rbma)

In [ ]:
spec, target, annotation = next(loader)
model.eval()
with torch.no_grad():
    output = model(spec)

In [ ]:
print(output.shape)
print(target.shape)
print(spec.shape)

In [ ]:
section = range(1000, 20000)
plt.figure(figsize=(10, 10))
plt.plot(output[0][0][section].numpy(), label="Output")
plt.legend()
plt.show()

In [ ]:
section = range(1000, 20000)
plt.figure(figsize=(10, 10))
plt.plot(output[0][0][section].numpy(), label="Output")
plt.plot(target[0][0][section].numpy(), label="Target")
plt.legend()
plt.show()

In [ ]:
from model import ResidualBlock1d  # , CausalConv2d, ResidualBlock

eps = np.nextafter(np.float16(0.0), np.float16(1.0))
class_idx = 2
offset = 1
first_label = target[0][class_idx].nonzero()[15]
print(first_label)

causal_test = spec.clone()
causal_test[0, :, :first_label + offset] = 0
# causal_test[0, :, first_label+offset+30:] = 0
empty_model = ResidualBlock1d(1, 1, kernel_size=5)

model.eval()
with torch.no_grad():
    # output = model(causal_test)
    empty_output = empty_model(causal_test.unsqueeze(1)).squeeze(1)

output = empty_output

print(torch.min(output[0, class_idx, first_label - offset - 40:first_label + offset]))
print(torch.max(output[0, class_idx, first_label - offset - 40:first_label + offset]))
print(torch.min(output[0, class_idx, first_label + offset:]))
print(torch.max(output[0, class_idx, first_label + offset:]))

plt.figure(figsize=(10, 10))
plt.plot(output[0][0][first_label + offset - 10 - 1:first_label + offset + 10].numpy(), label="Output")
plt.minorticks_on()

# Test spectogram alignment

In [ ]:
import nnAudio.features
from dataset.RBMA13 import RBMA13
from dataset import load_audio
import torchaudio
import librosa
from dataset import get_time_index, get_indices

In [ ]:
rbma = RBMA13(
    path="./data/rbma_13",
    settings=dataset_settings,
    use_dataloader=False,
    is_train=False,
)

In [ ]:
mel_log, labels, gt = rbma[2]
test_audio = load_audio(rbma.get_full_path("RBMA-13-Track-03"), audio_settings.sample_rate, audio_settings.normalize)
gt = gt[2:]

In [ ]:
cqt = nnAudio.features.CQT1992v2(
    sr=audio_settings.sample_rate,
    hop_length=audio_settings.hop_size,
    fmin=20,
    fmax=20000,
    bins_per_octave=12,
    pad_mode=audio_settings.pad_mode,
)

nnAudio_stft = nnAudio.features.STFT(
    n_fft=audio_settings.fft_size,
    hop_length=audio_settings.hop_size,
    win_length=audio_settings.fft_size // 2,
    window="hann",
    center=audio_settings.center,
    pad_mode=audio_settings.pad_mode,
    trainable=False,
    output_format="Magnitude",
)

torch_stft = torchaudio.transforms.Spectrogram(
    n_fft=audio_settings.fft_size,
    hop_length=audio_settings.hop_size,
    win_length=audio_settings.fft_size // 2,
    power=2,
    center=audio_settings.center,
    pad_mode=audio_settings.pad_mode,
    normalized=True,
    onesided=True,
)

mel = torchaudio.transforms.MelScale(
    n_mels=audio_settings.n_mels,
    sample_rate=audio_settings.sample_rate,
    f_min=audio_settings.mel_min,
    f_max=audio_settings.mel_max,
    n_stft=audio_settings.fft_size // 2 + 1,
)

In [ ]:
cqt_spec = cqt(test_audio)[0]
torch_stft_spec = torch_stft(test_audio)
nnAudio_stft_spec = nnAudio_stft(test_audio)
mel_spec = mel(torch_stft_spec)
print(torch_stft_spec.shape)
print(nnAudio_stft_spec.shape)
print(cqt_spec.shape)
print(mel_spec.shape)

In [ ]:
print(torch_stft_spec.shape[-1] / audio_settings.sample_rate * audio_settings.hop_size)
print(nnAudio_stft_spec.shape[-1] / audio_settings.sample_rate * audio_settings.hop_size)
print(cqt_spec.shape[-1] / audio_settings.sample_rate * audio_settings.hop_size)
print(test_audio.shape[-1] / audio_settings.sample_rate)
print(audio_settings.fft_size // audio_settings.hop_size)

In [ ]:
plot_range = slice(000, 200)

gt_labels = [np.array(
    [label - plot_range.start / 100 for label in cls if plot_range.start <= int(label * 100) <= plot_range.stop]) for
    cls in gt]
print(gt_labels)

db = librosa.amplitude_to_db(cqt_spec[:, plot_range], ref=np.max)
fig, ax = plt.subplots(nrows=3)
_img1 = librosa.display.specshow(
    db,
    y_axis='cqt_hz',
    x_axis='time',
    ax=ax[0],
    sr=audio_settings.sample_rate,
    hop_length=audio_settings.hop_size,
    fmin=20,
    fmax=20000,
)
ax[0].set(title='CQT spectrogram')

M = mel_spec[:, plot_range].detach().numpy()
M_db = librosa.power_to_db(M, ref=np.max)
_img2 = librosa.display.specshow(
    M_db,
    y_axis='mel',
    x_axis='time',
    ax=ax[1],
    sr=audio_settings.sample_rate,
    hop_length=audio_settings.hop_size,
    fmin=audio_settings.mel_min,
    fmax=audio_settings.mel_max
)
ax[1].set(title='Mel spectrogram')

time = get_time_index(plot_range.stop - plot_range.start, audio_settings.sample_rate, audio_settings.hop_size)
time = np.array([time] * annotation_settings.n_classes).T

ax[0].vlines(time[labels[:, plot_range].T == 1], ymin=0, ymax=1)
ax[1].vlines(time[labels[:, plot_range].T == 1], ymin=0, ymax=1)
ax[2].vlines(time[labels[:, plot_range].T == 1], ymin=0, ymax=1, colors=["blue"])
for cls in gt_labels:
    ax[2].vlines(cls, ymin=0, ymax=1, colors=["red"])
ax[2].set_xlim(left=0, right=time[-1][0])
plt.tight_layout()

In [ ]:
sample_rate = audio_settings.sample_rate
hop_size = audio_settings.hop_size
fft_size = audio_settings.fft_size

for cls in gt_labels:
    if len(cls) == 0:
        continue
    index = get_indices(cls, sample_rate, hop_size)
    time = get_time_index(np.max(index) + 1, sample_rate, hop_size)
    assert np.allclose(time[index], cls,
                       atol=hop_size / sample_rate / 2), f"{time[index][np.abs(time[index] - cls) > hop_size / sample_rate / 2]} \n{cls[np.abs(time[index] - cls) > hop_size / sample_rate / 2]}"
    print()


# Test audio loading

In [ ]:
%matplotlib inline
import librosa
from dataset import load_audio, get_length
import torchaudio
import numpy as np
import torch
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
def plot_audio(audio, sr, title="", offset=0, num_samples=None):
    if num_samples is None:
        num_samples = len(audio)
    t = np.arange(offset, num_samples + offset) / sr
    plt.figure(figsize=(10, 5))
    plt.plot(t, audio[offset:offset + num_samples])
    plt.title(title)
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()

In [ ]:
audio_path = "./data/a2md_public/ytd_audio/dist0p00/ytd_audio_00040_TRVNNMF128F4285161.mp3"
sample_rate = librosa.get_samplerate(audio_path)
duration = get_length(audio_path)
print(sample_rate, duration)
segment = (12, 20)
print(segment[0] * sample_rate, segment[1] * sample_rate)

In [ ]:
import timeit

n_runs = 10

torch_time = timeit.timeit(
    lambda: torchaudio.load(audio_path, frame_offset=int(segment[0] * sample_rate),
                            num_frames=int((segment[1] - segment[0]) * sample_rate), backend="ffmpeg", normalize=True),
    number=n_runs
)

librosa_time = timeit.timeit(
    lambda: librosa.load(audio_path, sr=sample_rate, offset=segment[0], duration=segment[1] - segment[0]),
    number=n_runs
)

loader_time = timeit.timeit(
    lambda: load_audio(path=audio_path, sample_rate=sample_rate, start=segment[0], end=segment[1], normalize=False,
                       backend="sox"),
    number=n_runs
)

print(f"Loader: {loader_time:.4f} s")
print(f"Torch: {torch_time:.4f} s")
print(f"Librosa: {librosa_time:.4f} s")

In [ ]:
torch_audio = %time torchaudio.load(audio_path, frame_offset=int(segment[0] * sample_rate), num_frames=int((segment[1] - segment[0]) * sample_rate), normalize=True)[0].squeeze()
librosa_audio, _ = %time librosa.load(audio_path, sr=sample_rate, offset=segment[0], duration=segment[1] - segment[0])
loader_audio = %time load_audio(path=audio_path, sample_rate=sample_rate, start=segment[0], end=segment[1], normalize=False)

In [ ]:
ref_index = np.argmax(np.abs(librosa_audio))
other_ref_index = torch.argmax(torch.abs(torch_audio)).item()
if ref_index != other_ref_index:
    print(ref_index, other_ref_index)

index = torch.argmax(torch.abs(loader_audio)).item()
actual_offset = index - ref_index
print(actual_offset)
print(actual_offset / sample_rate)

In [ ]:
print(loader_audio.shape, loader_audio.dtype)
print(torch_audio.shape, torch_audio.dtype)
print(librosa_audio.shape, librosa_audio.dtype)

In [ ]:
print(torch.max(loader_audio).item(), torch.min(loader_audio).item())
print(torch.max(torch_audio).item(), torch.min(torch_audio).item())
print(np.max(librosa_audio), np.min(librosa_audio))
print()

difference = loader_audio.shape[-1] - torch_audio.shape[-1]
if difference != 0:
    print(f"Loader audio is longer by {difference} samples")
    # loader_audio = loader_audio[difference:]

if not np.allclose(librosa_audio, torch_audio.numpy(), atol=1e-5):
    print(f"Librosa and torch audio are not equal")
    print(np.max(np.abs(librosa_audio - torch_audio.numpy())))
if not np.allclose(loader_audio.numpy(), torch_audio.numpy(), atol=1e-5):
    print(f"Loader and torch audio are not equal")
    print(np.max(np.abs(loader_audio.numpy() - torch_audio.numpy())))
if not np.allclose(loader_audio, librosa_audio, atol=1e-5):
    print(f"Loader and librosa audio are not equal")
    print(np.max(np.abs(loader_audio.numpy() - librosa_audio)))

In [ ]:
abs_error = 1e-4
mask = (~(np.isclose(loader_audio.numpy(), librosa_audio, atol=abs_error))).nonzero()
print(mask, len(mask[0]))
errors = loader_audio[~np.isclose(loader_audio.numpy(), librosa_audio, atol=abs_error)]
print(errors.shape, errors.shape[-1] / loader_audio.shape[-1])

In [ ]:
if difference > 0:
    extra_audio = loader_audio[:difference]
    extra_audio = extra_audio / torch.max(extra_audio)
    ipd.Audio(extra_audio, rate=sample_rate)

In [ ]:
num_samples = 800
offset = 0

In [ ]:
plot_audio(loader_audio, sample_rate, "Loader audio", num_samples=num_samples, offset=offset)
plot_audio(torch_audio.numpy(), sample_rate, "Torch audio", num_samples=num_samples, offset=offset)
plot_audio(librosa_audio, sample_rate, "Librosa audio", num_samples=num_samples, offset=offset)

In [ ]:
ipd.Audio(loader_audio.numpy(), rate=sample_rate)

# Dataset inspection

In [ ]:
from pretty_midi import PrettyMIDI
import pretty_midi
import numpy as np
from dataset.A2MD import get_tracks

In [ ]:
midi = PrettyMIDI("./data/a2md_public/align_mid/dist0p00/align_mid_00012_TRYIYUF128F932573D.mid")

In [ ]:
def get_drum_pitch_velocity(_midi: PrettyMIDI) -> np.ndarray:
    drum_instruments: list[pretty_midi.Instrument] = [
        instrument for instrument in _midi.instruments if instrument.is_drum
    ]
    notes = np.array(
        [
            (note.pitch, note.velocity)
            for instrument in drum_instruments
            for note in instrument.notes
        ]
    )
    return notes


np.array(drum_instruments[0])
# np.unique(notes[:, 0].astype(int), return_counts=True)

# Inspect trained model

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataset.RBMA13 import RBMA13
from dataset.A2MD import A2MD
from settings import DatasetSettings, CNNMambaSettings, TrainingSettings, CNNSettings, CNNAttentionSettings
from dataclasses import asdict
import dataset
from evallib import peak_pick_max_mean, calculate_pr

from model.CRNN import CRNN
from model.unet import UNet
from model.cnnM2 import CNNMambaFast
from model.cnnM import CNNMamba
from model.cnnA import CNNAttention

In [ ]:
checkpoint = torch.load("../models/Jun04_18-08-56_marclie-desktop_mamba_fast_66.71.pt", map_location=torch.device("cpu"))

In [ ]:
torch.cuda.empty_cache()
dataset_settings = DatasetSettings.from_flat_dict(checkpoint["dataset_settings"])
dataset_settings.segment_type = None
training_settings = TrainingSettings.from_flat_dict(checkpoint["training_settings"])
model_settings = training_settings.get_model_settings_class().from_flat_dict(checkpoint["model_settings"])
match training_settings.model_settings:
    case "cnn":
        model = CNN(**asdict(model_settings), n_classes=dataset_settings.annotation_settings.n_classes,
                    n_mels=checkpoint["dataset_settings"]["n_mels"])
    case "cnn_attention":
        model = CNNAttention(**asdict(model_settings), n_classes=dataset_settings.annotation_settings.n_classes,
                             n_mels=checkpoint["dataset_settings"]["n_mels"])
    case "mamba":
        model = CNNMamba(**asdict(model_settings), n_classes=dataset_settings.annotation_settings.n_classes,
                         n_mels=checkpoint["dataset_settings"]["n_mels"])
    case "mamba_fast":
        model = CNNMambaFast(**asdict(model_settings), n_classes=dataset_settings.annotation_settings.n_classes,
                             n_mels=checkpoint["dataset_settings"]["n_mels"])
    case "unet":
        model = UNet(**asdict(model_settings))
    case "crnn":
        model = CRNN(**asdict(model_settings), n_classes=dataset_settings.annotation_settings.n_classes,
                     n_mels=checkpoint["dataset_settings"]["n_mels"])
    case _:
        raise ValueError("Invalid model setting")
model.load_state_dict(checkpoint["model"])
model = model.eval()

In [ ]:
training_settings

In [ ]:
rbma = RBMA13(
    path="../data/rbma_13",
    settings=dataset_settings,
    use_dataloader=True,
    is_train=False,
    segment=False,
)

rbma_loader = dataset.get_dataloader(rbma, 1, 1, is_train=False)

# a2md = A2MD(
#     path="data/a2md_public",
#     settings=dataset_settings,
#     use_dataloader=False,
#     is_train=False,
#     segment=False,
# )

In [ ]:
mel_log, labels, gt = next(iter(rbma_loader))
print(mel_log.shape, labels.shape, len(gt[0]))
gt = gt[0][2:]

In [ ]:
next(iter(rbma_loader))

In [ ]:
model = model.cuda()
output: torch.Tensor = model(mel_log.cuda()).detach().cpu().sigmoid().squeeze(0)
print(output.shape)

In [ ]:
peaks = peak_pick_max_mean(output.unsqueeze(0), dataset_settings.audio_settings.sample_rate, dataset_settings.audio_settings.hop_size, dataset_settings.audio_settings.fft_size)
for cls in peaks:
    for peak in cls:
        peak[0] -= dataset_settings.annotation_settings.time_shift
_, _, _, score_sum, score_avg, _ = calculate_pr(peaks, [gt], detection_window=0.025)
score_avg, score_sum

In [ ]:
model = model.cuda()
with torch.inference_mode():
    segment = mel_log[..., 6200:7000]
    print(segment.min(), segment.max())
    segment = segment.cuda()
    # for some reason, the output is dependent on the segment length
    expected_output = model(segment)[..., :400].detach().cpu().squeeze(0)
    # plt.plot(expected_output[0].numpy(), scaley=False)
    for i in range(100):
        # segment = (torch.rand_like(segment) * 8 + 1).log()
        random = (torch.rand_like(segment) * 8 + 1).log()
        in_seg = torch.concatenate([segment, random], dim=-1)
        out_mixed = model(in_seg)[..., :400].detach().cpu().squeeze(0)
        # plt.plot((out_mixed - expected_output).abs()[0].numpy(), scaley=True)
        # assert torch.allclose(expected_output, out_clean, atol=1e-7), f"{(out_clean - expected_output).abs().sum().item(), (out_clean - expected_output).argmax(dim=-1)}"
        assert torch.allclose(out_mixed, expected_output,
                              atol=1e-9), f"{(out_mixed - expected_output).abs().sum().item(), (out_mixed - expected_output).argmax(dim=-1)}"

In [ ]:
gt_labels = dataset.get_labels(gt, dataset_settings.audio_settings.sample_rate, dataset_settings.audio_settings.hop_size, labels.shape[-1])
plt.plot(output[0, 7230:7230 + 10].numpy(), scaley=False)
plt.plot(labels[0, 0, 7230:7230 + 10].numpy(), scaley=False)
plt.plot(gt_labels[0, 7230:7230 + 10].numpy(), scaley=False)

In [ ]:
num_pos, num_neg = a2md.get_sample_distribution()
# weights according to https://markcartwright.com/files/cartwright2018increasing.pdf section 3.4.1 Task weights
total = (num_pos + num_neg)[0]
print(num_pos)
p_i = num_pos / (total * dataset_settings.annotation_settings.n_classes)
weight = 1 / (-p_i * p_i.log() - (1 - p_i) * (1 - p_i).log())
weight

# Test Causal Blocks

In [ ]:
import torch
from model import CausalAvgPool1d, CausalConv1d, CausalConv2d, CausalMaxPool1d, ResidualBlock
from model.cnn_feature import CNNFeature
from functools import partial

In [ ]:
blocks_to_test = [
    partial(torch.nn.Conv2d, 100, 32, 1),
    partial(torch.nn.MaxPool2d, kernel_size=(2, 1), stride=(2, 1)),
    partial(torch.nn.Dropout2d, 0.3),
    partial(ResidualBlock, 100, 10, 3),
    partial(CNNFeature, num_channels=16, n_layers=2, down_sample_factor=2, channel_multiplication=2, activation=torch.nn.ReLU(), causal=True, dropout=0.3, in_channels=100)
]

In [ ]:
torch.use_deterministic_algorithms(True)
compare_size = 64
for block in blocks_to_test:
    mod = block().eval()
    print(f"Testing: {mod.__class__}")
    for name, param in mod.named_parameters():
        # Set weights to one
        if 'weight' in name:
              param.data = torch.ones_like(param)
        elif 'bias' in name:
              param.data = torch.ones_like(param)
    with torch.inference_mode():
        for i in range(100):
            a = torch.rand(1, 100, 10, compare_size)
            b = torch.concatenate([a, torch.rand_like(a)[..., :]], dim=-1)
            assert (mod(a)[..., :compare_size] == mod(b)[..., :compare_size]).all(), f"Error in iteration {i}: {(mod(a)[..., :compare_size] - mod(b)[..., :compare_size]).abs().sum()}"

# Learning rate finder

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataclasses import asdict


from dataset.datasets import get_dataset
from hyperparameters import final_experiment_params
from settings import Config
from model.CRNN import CRNN
from model.unet import UNet
from model.cnnM2 import CNNMambaFast
from model.cnnM import CNNMamba
from model.cnnA import CNNAttention

from torch_lr_finder import LRFinder, TrainDataLoaderIter, ValDataLoaderIter
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


In [ ]:
import os

os.chdir("..")
# get the current working directory
current_working_directory = os.getcwd()
current_working_directory

In [ ]:
class CustomTrainIter(TrainDataLoaderIter):
    def inputs_labels_from_batch(self, batch_data):
        return batch_data[0], batch_data[1]

class CustomValIter(ValDataLoaderIter):
    def inputs_labels_from_batch(self, batch_data):
        return batch_data[0], batch_data[1]

In [ ]:
config = Config.from_flat_dict(final_experiment_params["Mamba fast"])

model_settings = config.model
match config.training.model_settings:
    case "cnn":
        model = CNN(**asdict(model_settings), n_classes=config.dataset.annotation_settings.n_classes,
                    n_mels=config.dataset.audio_settings.n_mels)
    case "cnn_attention":
        model = CNNAttention(**asdict(model_settings), n_classes=config.dataset.annotation_settings.n_classes,
                             n_mels=config.dataset.audio_settings.n_mels)
    case "mamba":
        model = CNNMamba(**asdict(model_settings), n_classes=config.dataset.annotation_settings.n_classes,
                         n_mels=config.dataset.audio_settings.n_mels)
    case "mamba_fast":
        model = CNNMambaFast(**asdict(model_settings), n_classes=config.dataset.annotation_settings.n_classes,
                             n_mels=config.dataset.audio_settings.n_mels)
    case "unet":
        model = UNet(**asdict(model_settings))
    case "crnn":
        model = CRNN(**asdict(model_settings), n_classes=config.dataset.annotation_settings.n_classes,
                     n_mels=config.dataset.audio_settings.n_mels)
    case _:
        raise ValueError("Invalid model setting")

In [ ]:
config.training.dataset_version = "S"
config.training.test_sets = ()

train_loader, val_loader, _ = get_dataset(config.training, config.dataset)

In [ ]:
model = model.cuda()

optimizer = optim.RAdam(
    model.parameters(),
    lr=config.training.learning_rate,
    betas=(config.training.beta_1, config.training.beta_2),
    weight_decay=config.training.weight_decay,
    decoupled_weight_decay=config.training.decoupled_weight_decay,
    eps=config.training.epsilon,
)
criterion = nn.BCEWithLogitsLoss()

amp_config = {
    'device_type': 'cuda',
    'dtype': torch.float16,
}
grad_scaler = torch.amp.GradScaler("cuda")


In [ ]:
lr_finder = LRFinder(
    model, optimizer, criterion, device='cuda',
    amp_backend='torch', amp_config=amp_config, grad_scaler=grad_scaler
)
lr_finder.range_test(train_loader, val_loader=val_loader, start_lr=1e-6, end_lr=0.1, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
plt.plot(range(10))
plt.show()